0. Create a Model with division, death, and volume

In [ ]:
%matplotlib inline
from bioscrape.lineage import LineageModel
from bioscrape.lineage import LineageVolumeSplitter
import numpy as np
import pylab as plt

k = 1
d = .1
g = .2
rxn1 = (["A"], ["A", "X"], "massaction", {"k":k})
rxn2 = (["A"], ["B"], "massaction", {"k":d})
rxn3 = (["B"], ["A"], "massaction", {"k":d})
rxns = [rxn1, rxn2, rxn3]
LM1 = LineageModel(reactions = rxns, initialize_model = True, initial_condition_dict = {"X": 0, "A":1})
LM1.create_volume_event("linear volume", {"growth_rate":g}, "massaction", {"k":.1, "species":"X"})
vsplit = LineageVolumeSplitter(LM1, options = {"A":"duplicate", "B":"duplicate"})
LM1.create_division_rule("deltaV", {"threshold":5}, vsplit)
LM1.create_division_rule("time", {"threshold":50}, vsplit)
LM1.create_death_event("death", {}, "hillpositive", {"s1":"X", "K":25, "n":2, "k":.25})
LM1.py_initialize()

1. Testing SimulateCellLineage in the LineageSSASimulator

In [ ]:
from bioscrape.lineage import py_SimulateCellLineage

max_time = 200
timepoints = np.linspace(0,max_time,max_time*100)

lineage = py_SimulateCellLineage(timepoints, Model = LM1)

sch_tree = [[]]
sch_tree_length = 1
for i in range(lineage.py_size()):
	sch = lineage.py_get_schnitz(i)
	if sch.py_get_parent() == None:
		sch_tree[0].append(sch)
	else:
		for j in range(len(sch_tree)):
			parent = sch.py_get_parent()
			if parent in sch_tree[j]:
				if len(sch_tree)<= j+1:
					sch_tree.append([])
					sch_tree_length += 1
				sch_tree[j+1].append(sch)

color_list = []
for i in range(sch_tree_length):
	color_list.append((i/sch_tree_length, 0, 1.-i/sch_tree_length))

X_ind = LM1.get_species_index('X')
plt.figure()
plt.subplot(211)
count = 0
for i in range(sch_tree_length):
    for sch in sch_tree[i]:
        count+=1
        plt.plot(sch.py_get_time(), sch.py_get_data()[:,X_ind], color = color_list[i])


plt.subplot(212)
for i in range(sch_tree_length):
	for sch in sch_tree[i]:
		plt.plot(sch.py_get_time(), sch.py_get_volume(), color = color_list[i])

print("Total Cells Simulated", count)

plt.show()


Testing the LineageSSASimulator PropogateCells Function

In [ ]:
%matplotlib inline

from bioscrape.lineage import py_PropagateCells

#max_time = 50
timepoints = np.linspace(0,max_time,max_time*100)

final_states = py_PropagateCells(timepoints, Model = LM1)
X_ind = LM1.get_species_index('X')
x_counts = []
volumes=[]
for fs in final_states:
    time, vol, counts = fs.__getstate__()
    x_counts.append(counts[X_ind])
    volumes.append(vol)
plt.figure()
plt.subplot(211)
plt.hist(x_counts, label = "Count X")
plt.legend()
plt.subplot(212)
plt.hist(volumes, label = "Volume")
plt.legend()

plt.show()

3. Testing Single Cell Lineage Simulation

In [ ]:
from bioscrape.lineage import py_SingleCellLineage

#max_time = 250
timepoints = np.linspace(0,max_time,max_time*100)
final_lineage = py_SingleCellLineage(timepoints, Model = LM1)

lineage_size = len(final_lineage)
X_ind = LM1.get_species_index('X')
x_counts = []
volumes=[]
plt.figure()
color_list = [(i/lineage_size, 0, 1-i/lineage_size) for i in range(lineage_size)]
for i in range(lineage_size):
    f = final_lineage[i]
    tp = f.py_get_timepoints()
    r = f.py_get_result()
    v = f.py_get_volume()
    x = r[:, X_ind]
    
    plt.subplot(121)
    plt.plot(tp, x, color = color_list[i])
    plt.subplot(122)
    plt.plot(tp, v, ":", color = color_list[i])

plt.show()